##### Copyright 2023 Google LLC

In [12]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

In [2]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [4]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [5]:
model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicGFydHMiOlt7InRleHQiOiJpbnB1dDogIn0seyJmaWxlX2RhdGEiOnsibWltZV90eXBlIjoiYXBwbGljYXRpb24vb2N0ZXQtc3RyZWFtIiwiZHJpdmVfaWQiOiIxRlBYd0VPY3NDSE1EU09LQzB4d09qdktIY3dlN0dYTm8ifX0seyJ0ZXh0Ijoib3V0cHV0OiDrqZTribTrqoU6IOudvOuptFxcblxcbjEuIOyerOujjFxcbuudvOuptCAx67SJ7KeAXFxu7Iqk7ZSEIDEvMuqwnFxcbuuMgO2MjCAx67+M66asXFxu7Jis66as67iM7JygIDLsiqTtkbxcXG5cXG4qIOyhsOumrOyInOyEnFxcbjEuIO2PieyGjOuMgOuhnCDrrLzsnYQgNTUwbWwg64Sj6rOgIOuBk+yXrOyjvOyEuOyalCDCoCDCoFxcbjIuIOuBk+uKlCDrrLzsl5Ag652866m06rO8IOyKpO2UhOulvCDrhKPslrTso7zshLjsmpQg7Jes6riw7IScIO2PrOyduO2KuOuKlCDrnbzrqbTsnYQgNzB+ODAl66eMIOydte2YgOyjvOyLnOuKlCDqsbDsl5DsmpQs7J2065Sw6rCAIOudvOuptCDrs7bsnLzrqbTshJwg65iQIOydte2YgCDspITqsbDquLAg65WM66y47JeQIO2RuSDsnbXtnojrqbQg7Y287KeIIOyImOqwgCDsnojslrTsmpR+Ie2PrOyduO2KuOuKlCDrnbzrqbTsnYQgNzB+ODAl66eMIOydte2eiOq4sFxcbjMuIOyCtOynnSDrjZwg7J217J2AIOudvOuptCDrrLzsnYQg67KE66Ck7KO87IS47JqULiDqt7gg65WMIOudvOuptCDqta3rrLwgM3407Iqk7ZG87J2AIOuCqOqyqOyjvOyLnOqzoCDrsoTroKTso7zshLjsmpReXiDCoCDCoFxcbjQuIOydtOygnCDtjKzsl5Ag7Jis66as67iM7JygIG9yIOyLneyaqeycoCAy7Iqk7ZG87J2EIOyYrOugpOyjvOyEuOyalCBcXG41LiDri6TsnYwg7JWE6rmMIOyEpOydteydgCDrnbzrqbTsnYQg64Sj6rOgIOuztuyVhOykhOqxsOyXkOyalC4g6re466as6rOgIOyVhOq5jCDrnbzrqbQg64GT64qUIOusvCAzfjTsiqTtkbwg6riw7Ja164KY7Iuc7KOgP15eXFxuNi4g64GT7J24IOusvCA07Iqk7ZG87J2EIO2VqOq7mCDrhKPslrTshJwg7JW9IOu2iOyXkOyEnCDrs7bslYTso7zshLjsmpR+IMKgXFxuNy4g64uk7J2MIOuMgO2MjOulvCDsmKzroKTso7zsi5zqtazsmpR+flxcbjguIOuMgO2MjO2WpeydtCDshpTshpR+fiDrgqDrlYzquYzsp4Ag67O27JWE7KO87IS47JqU7KCA64qUIOuwseyiheybkCDroIjsi5ztlLzspJHsl5Ag6rCA7J6lIOuniOydjOyXkCDrk5zripTqsowg64yA7YyM7Zal7J246rGwIOqwmeyVhOyalCDjhY7jhY7sm5Drnpgg7YyM66W8IOyii+yVhO2VmOq4sOuPhCDtlZjsp4Drp4wg64yA7YyM7ZalIOyGlOyGlCDrgpjripTqsowg7LC4IOyii+yVhOyalCBcXG45LiDri6TsnYwg652866m07Iqk7ZSEIDEvMuydhCDrhKPqs6Ag7IOk7IOk7IOlfiDrs7bslYTso7zshLjsmpRcXG4xMC4g7KGw6riIIOuNlCDrhKPsnLzrqbQg7Zi57IucIOynpOyImOqwgCDsnojsnLzri4gg65Oc7IWU67O07Iuc66m07IScIOyhsOygiO2VmOuKlOqxuOuhnH5eXiDCoCDCoOynoH4hIOyZhOyEseuQmOyXiOyWtOyalCwifSx7InRleHQiOiJpbnB1dDogIn0seyJmaWxlX2RhdGEiOnsibWltZV90eXBlIjoiYXBwbGljYXRpb24vb2N0ZXQtc3RyZWFtIiwiZHJpdmVfaWQiOiIxSEFlR3hMai1CSURjRVFheXp5clUzUXBtdEdZTlJ5S0sifX0seyJ0ZXh0Ijoib3V0cHV0OiAifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [6]:
gais_contents

[{'parts': [{'text': 'input: '},
   {'file_data': {'mime_type': 'application/octet-stream',
     'drive_id': '1FPXwEOcsCHMDSOKC0xwOjvKHcwe7GXNo'}},
   {'text': 'output: 메뉴명: 라면\\n\\n1. 재료\\n라면 1봉지\\n스프 1/2개\\n대파 1뿌리\\n올리브유 2스푼\\n\\n* 조리순서\\n1. 평소대로 물을 550ml 넣고 끓여주세요 \xa0 \xa0\\n2. 끓는 물에 라면과 스프를 넣어주세요 여기서 포인트는 라면을 70~80%만 익혀주시는 거에요,이따가 라면 볶으면서 또 익혀 줄거기 때문에 푹 익히면 퍼질 수가 있어요~!포인트는 라면을 70~80%만 익히기\\n3. 살짝 덜 익은 라면 물을 버려주세요. 그 때 라면 국물 3~4스푼은 남겨주시고 버려주세요^^ \xa0 \xa0\\n4. 이제 팬에 올리브유 or 식용유 2스푼을 올려주세요 \\n5. 다음 아까 설익은 라면을 넣고 볶아줄거에요. 그리고 아까 라면 끓는 물 3~4스푼 기억나시죠?^^\\n6. 끓인 물 4스푼을 함께 넣어서 약 불에서 볶아주세요~ \xa0\\n7. 다음 대파를 올려주시구요~~\\n8. 대파향이 솔솔~~ 날때까지 볶아주세요저는 백종원 레시피중에 가장 마음에 드는게 대파향인거 같아요 ㅎㅎ원래 파를 좋아하기도 하지만 대파향 솔솔 나는게 참 좋아요 \\n9. 다음 라면스프 1/2을 넣고 샤샤샥~ 볶아주세요\\n10. 조금 더 넣으면 혹시 짤수가 있으니 드셔보시면서 조절하는걸로~^^ \xa0 \xa0짠~! 완성되었어요,'},
   {'text': 'input: '},
   {'file_data': {'mime_type': 'application/octet-stream',
     'drive_id': '1HAeGxLj-BIDcEQayzyrU3QpmtGYNRyKK'}},
   {'text': 'output: '}]}]

In [7]:
generation_config

{'temperature': 1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192}

In [8]:
safety_settings

{}

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [9]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive_id is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [13]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

input:  



UnboundLocalError: local variable 'drive' referenced before assignment

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [11]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [ ]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [ ]:
contents

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

In [ ]:
response.candidates

## Or Create a chat

In [ ]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [ ]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))